# Chapter 9: Pythonic Objects

Python's data model allows user-defined types to behave as naturally as the built-in types. This can be accomplished without inheritance, in the spirit of [duck typing](https://en.wikipedia.org/wiki/Duck_typing): you implement the methods needed for your objects to behave as expected.

This chapter shows how to implement several special methods that are commonly seen in Python objects of many different types.

We will see how to:

* Support the built-in functions that produce alternative object representation.

* Implement an alternative constructor as a class method.

* Extend the format mini-language used by the `format()` built-in and `str.format()` method.

* Provide read-only access to attributes.

* Make an object hashable for use in sets and `dict` keys.

* Save memory by using `__slots__`.

Through examples we will also examine:

* How and when to use the `@classmethod` and `@staticmethod` decorators.

* Private and protected attributes in Python: usage, conventions, and methods.

## Object Representations

Recall that Python has two standard ways of getting a string representation from an object:

1. `repr()`: Return a string representing the object as the **developer** wants to see it.

2. `str()`: Return a string representing the object as the **user** wants to see it.

The special methods `__repr__` and `__str__` support `repr()` and `str()`.

There are two additional special methods to support alternative string representations:

3. `__bytes__`: Return the byte string representation of the object (invoked by `bytes()`).

4. `__format__`: Display object using special formatting codes (invoked by `format()` and `str.format()`.

## Vector Class Redux

To demonstrate different ways of generating object representations, we will extend the vector class from [chapter 1]() to two dimensions.

In [1]:
# Example 9-2. vector2d_v0.py: Implementation of Vector2d using only special methods

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    

## An Alternative Constructor

Since we can export `Vector2d` as bytes, it makes sense to implement functionality to go the other way; we need a method to import `Vector2d` from a binary sequence.

In [2]:
# Example 9.3. vector2d_v1.py: Implementation of Vector2d allowing conversions from bytes to Vector2d instance.

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)


## `classmethod` Versus `staticmethod`

`classmethod` is used to define a method that operates on the class and not on instances. 

`classmethod` changes the way the method is called, so it **receives the class itself as the first argument, instead of an instance**.

Its most common use is for alternative constructors, like `frombytes` in example 9-3. Note how the last line uses the `cls` argument by invoking it to build a new instance.

By convention, the first parameter of a class method should be named `cls`.

The `staticmethod` decorator changes a method so that it receives no special first argument.

**In essence, a static method is just like a plain function that happens to live in a class body, instead of being defined at the module level**.

The next example contrast the operation of `classmethod` and `staticmethod`.

In [3]:
# Example 9-4. Comparing behavious of classmethod and staticmethod.

class Demo:
    """
    klassmeth returns all positional arguments.
    statmeth does the same.
    """
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

# No matter how it is invoked, `klassmeth` receives the Demo class as first argument
print(Demo.klassmeth())
print(Demo.klassmeth("spam"))

# `statmeth` behave like a plain function
print(Demo.statmeth())
print(Demo.statmeth("spam"))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


Ramalho is of the opinion that `@staticmethod` is of little practical use. He argues that if you want a function that doesn't interact with the class, simply define it in the module.

Ramalho provides [this blog post](https://julien.danjou.info/guide-python-static-class-abstract-methods/) as a counter-argument to his own viewpoint.

## Formatted Displays

**Note:** Since Fluent Python was published, f-strings have been introduced. This is not covered in the book. Read more in [PEP 498](https://www.python.org/dev/peps/pep-0498/).

The `format()` built-in function and the `str.format()` method delegate the actual formatting to each type by calling their `.__format__(format_spec)` method.

`format_spec` is a formatting specifier, which is either:

1. The second argument in `format(my_obj, format_spec)`, or

2. Whatever appears after the colon in a replacement field delimited with `{}` inside a format string used with `str.format()`.

Note that the two approaches above use slightly different format specifiers. The first uses  the *Format Specification Mini-Language*, the second uses the *Format String Syntax*. [Detailed info is available in the docs](https://docs.python.org/3.8/library/string.html).

The *Format Specification Mini-Language* is extensible because each class gets to interpret the `format_spec` argument as it likes.

In the next chunk, we extend `Vector2d` by implementing `__format__`.

In [4]:
# Example 9.5. Vector2d.format method, take #1

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def __format__(self, fmt_spec=""):
        components = (format(coord, fmt_spec) for coord in self)
        return "({}, {})".format(*components)


In [8]:
v = Vector2d(2, 3)
format(v)

'(2.0, 3.0)'

In [9]:
format(v, ".2f")

'(2.00, 3.00)'

In [10]:
format(v, ".3e")

'(2.000e+00, 3.000e+00)'

Next, we add a custom formatting to our mini-language.

If the format specifier ends with a `p`, the vector will be displayed in polar coordinates `<r, theta>`.

The requires first generating the polar coordinates. The magnitude (`r`) is already implemented with `__abs__`. We need to add a function for calculating `theta`:

In [11]:
# Add this to Vector2d
def angle(self):
    return math.atan2(self.y, self.x)

In [2]:
# Example 9.5. Vector2d.format method, take #2 (w/ polar coordinates)

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("p"):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:
            coords = self
            outer_fmt = "({}, {})"
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    

In [15]:
v = Vector2d(1, 1)
format(v, "p")

'<1.4142135623730951, 0.7853981633974483>'

In [16]:
format(v, ".2fp")

'<1.41, 0.79>'

In [17]:
format(v, ".3ep")

'<1.414e+00, 7.854e-01>'

## A Hashable `Vector2d`

In order to build sets of `Vector2d` instances, we need them to be hashable. This requires implementing a `__hash__` method.

We also need to make vector instances immutable (see "What is Hashable?" on **p. 69**).

We start by making the `x` and `y` components *read-only* by using the `property` decorator (more on this in [chapter 19]()).

In [3]:
# Example 9.7. vector2d_v3.py: making Vector2d (reasonably) immutable

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes

    def __init__(self, x, y):
        self.__x = float(x) # Use exactly two leading underscores to make attribute private
        self.__y = float(y)
        
    @property # Marks the getter method of a property
    def x(self):
        return self.__x
    
    @property # Marks the getter method of a property
    def y(self):
        return self.__y

    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("p"):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:
            coords = self
            outer_fmt = "({}, {})"
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    

In [22]:
v = Vector2d(1, 1)
v.x

1.0

In [23]:
v.x = 2

AttributeError: can't set attribute

We can now implement the `__hash__` method by using the bitwise XOR operator `^` as [suggested in the docs](https://docs.python.org/3/reference/datamodel.html).

In [4]:
# Example 9.7. vector2d_v4.py: implementation of hash

from array import array
import math

class Vector2d:
    typecode = "d" # Used when converting Vector2d instances to/from bytes

    def __init__(self, x, y):
        self.__x = float(x) # Use exactly two leading underscores to make attribute private
        self.__y = float(y)
        
    @property # Marks the getter method of a property
    def x(self):
        return self.__x
    
    @property # Marks the getter method of a property
    def y(self):
        return self.__y

    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("p"):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:
            coords = self
            outer_fmt = "({}, {})"
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    

In [31]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
v3 = Vector2d(3, 4)

In [32]:
hash(v1), hash(v2), hash(v3)

(7, 384307168202284039, 7)

In [33]:
set([v1, v2, v3])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## Private and "Protected" Attributes in Python

There is no way to create private variables in Python, but there is a simple mechanism to avoid accidentally overwriting a "private" attribute in a subclass.

If an attribute is named in the form `__attribute`, with two leading underscores and zero or at most one trailing underscore, Python stores the the name in the instance `__dict__` **prefixed with a leading underscore and the class name**.

This feature is known as **name mangling**.

In [6]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [7]:
v1._Vector2d__x

3.0

Name mangling is about safety, not security. It is **designed to prevent accidental acces and not intentional wrongdoing**.

Some prefer to avoid the double underscore syntax and use just **one underscore prefix** to "protect" attributes by convention.

The single underscore prefix has no special meaning to the Python interpreter when used in attribute names, but there are very strong conventions that such attributes **should not be accessed outside the class**.

## Saving Space with the `__slots__` Class Attribute

By default, Python stores instance attributes in a per-instance `dict` named `__dict__`. 

As described in [chapter 3](), dictionaries have significant memory overhead due to the required hash tables.

If you are dealing with millions of instances with few attributes, the `__slots__` class attribute can save a lot of memory by storing the instance attributes in a `tuple` instead of a `dict`.

**Note**: A `__slots__` attribute inherited from a superclass has no effect. Python only takes into account `__slots__` attributes defined in each class individually.

**Note**: When `__slots__` is specified in a class, its instances will not be allowed to have any other attributes other than those named in `__slots__`.

**Note**: If you need instances to be targets of weak references, you need to add `__weakref__` to `__slots__`.

In [1]:
# Example 9.8. vector2d_v4_slots.py: slots attribute

from array import array
import math

class Vector2d:
    __slots__ = ("__x", "__y")
    typecode = "d" # Used when converting Vector2d instances to/from bytes

    def __init__(self, x, y):
        self.__x = float(x) # Use exactly two leading underscores to make attribute private
        self.__y = float(y)
        
    @property # Marks the getter method of a property
    def x(self):
        return self.__x
    
    @property # Marks the getter method of a property
    def y(self):
        return self.__y

    def __iter__(self): # Makes Vector2d iterable, allows unpacking
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return "{}({!r}, {!r})".format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=""):
        if fmt_spec.endswith("p"):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = "<{}, {}>"
        else:
            coords = self
            outer_fmt = "({}, {})"
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    

Running a memory test as shown in *example 9-12* yields the following results:

```bash
$ time python3 memtest.py vector2d_v3.py
Selected Vector2d type: vector2d_v3.Vector2d
Creating 10,000,000 Vector2d(3.0, 4.0) instances
Initial RAM usage:        7249920
  Final RAM usage:     1987338240
python3 memtest.py vector2d_v3.py  10.02s user 0.65s system 99% cpu 10.696 total

$ time python3 memtest.py vector2d_v3_slots.py
Selected Vector2d type: vector2d_v3_slots.Vector2d
Creating 10,000,000 Vector2d(3.0, 4.0) instances
Initial RAM usage:        7290880
  Final RAM usage:      684322816
python3 memtest.py vector2d_v3_slots.py  7.93s user 0.28s system 99% cpu 8.222 total
```

The `__slots__` version is both faster and has a significantly smaller RAM footprint.

## Overriding Class Attributes

A distinctive feature of Python is how class attributes can be used as default values for instance attributes.

For example, `Vector2d` has a `typecode` class attrubute. Because `Vector2d` instances are created without a `typecode` attribute of their own, the `self.typecode` will get the `Vector2d.typecode` attribute by default.

If you write to an instance attribute that doesn't exist (e.g. `typecode`), you create a *new instance attribute* and the class attribute of that same name is left untouched. From then on, the instance class attribute will be read.

The idiomatic Python way of setting a new class attribute is to subclass, as in the following example:

In [19]:
# Example 9-14. The ShortVector2d is a subclass of Vector2d, which only overwrite the default typecode

class ShortVector2d(Vector2d):
    typecode = "f"
    
sv = ShortVector2d(1/11, 1/27)
v = Vector2d(1/11, 1/27)
print(repr(sv), len(bytes(sv)))
print(repr(v), len(bytes(v)))

ShortVector2d(0.09090909090909091, 0.037037037037037035) 9
Vector2d(0.09090909090909091, 0.037037037037037035) 17
